In [ ]:
import pandas as pd
import re

def definitive_clean_and_aggregate_data(raw_data_path, output_path):
    """
    This function carefully cleans the data. It fills missing names from account_id,
    standardizes KPI names, removes useless KPIs, and aggregates the data
    by summing both yearly and monthly values.

    Args:
        raw_data_path (str): Path to the raw data CSV file.
        output_path (str): Path to save the cleaned master CSV file.

    Returns:
        pandas.DataFrame: The clean and prepared DataFrame.
    """
    print("--- Final Phase 1: Data Foundation (Correct Aggregation) ---")
    
    # 1. Load Data
    try:
        df = pd.read_csv(raw_data_path)
        print(f"Successfully loaded raw data from '{raw_data_path}'. Shape: {df.shape}")
    except FileNotFoundError:
        print(f"Error: File '{raw_data_path}' not found.")
        return None

    # 2. Correct Data Types
    print("\nStep 1: Correcting data types...")
    df['monthly_value'] = pd.to_numeric(df['monthly_value'], errors='coerce').fillna(0)
    df['yearly_value'] = pd.to_numeric(df['yearly_value'], errors='coerce').fillna(0)
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    print("Data types corrected.")

    # 3. Intelligently Fill Missing KPI Names
    print("\nStep 2: Carefully filling missing KPI names...")
    name_map = df.dropna(subset=['english_name']).groupby('account_id')['english_name'].first().to_dict()
    df['english_name'] = df['english_name'].fillna(df['account_id'].map(name_map))
    df['english_name'].fillna('Unknown KPI', inplace=True)
    print("Missing names filled based on account_id.")

    # 4. Deep Clean and Normalize All KPI Names
    print("\nStep 3: Performing deep cleaning on KPI names...")
    df['english_name_cleaned'] = df['english_name'].astype(str).str.strip().str.upper()
    df['english_name_cleaned'] = df['english_name_cleaned'].apply(lambda x: re.sub(r'[^A-Z0-9\s-]', '', x).strip())
    df['english_name_cleaned'] = df['english_name_cleaned'].apply(lambda x: re.sub(r'\s+', ' ', x))
    print("KPI names have been standardized.")

    # 5. Remove Useless and Invalid KPIs
    print("\nStep 4: Removing useless placeholder KPIs...")
    original_rows = len(df)
    df = df[~df['english_name_cleaned'].isin(['-', ''])]
    rows_removed = original_rows - len(df)
    print(f"{rows_removed} rows with placeholder KPI names have been removed.")

    # 6. Aggregate by summing monthly and yearly values
    print("\nStep 5: Aggregating data based on cleaned KPI names...")
    df_aggregated = df.groupby(['english_name_cleaned', 'year', 'month']).agg({
        'monthly_value': 'sum',
        'yearly_value': 'sum'  # This correctly preserves the yearly total from duplicates
    }).reset_index()
    print(f"Data has been aggregated. Original rows: {len(df)}, Aggregated rows: {len(df_aggregated)}")

    # 7. Confirm Final Number of Unique KPIs
    unique_kpi_count = df_aggregated['english_name_cleaned'].nunique()
    print(f"\nCONFIRMED: Found {unique_kpi_count} unique, valid KPIs after deep cleaning.")

    # 8. Create Date Column and Finalize Structure
    print("\nStep 6: Finalizing the dataset...")
    df_aggregated['date'] = pd.to_datetime(df_aggregated[['year', 'month']].assign(day=1))
    df_aggregated.rename(columns={'english_name_cleaned': 'english_name'}, inplace=True)
    final_df = df_aggregated[['date', 'english_name', 'monthly_value', 'yearly_value', 'year', 'month']]
    final_df.sort_values(by=['english_name', 'date'], inplace=True)
    final_df.reset_index(drop=True, inplace=True)
    print("Dataset has been finalized and sorted.")

    # 9. Save the Clean Master File
    try:
        final_df.to_csv(output_path, index=False)
        print(f"\n--- Phase 1 Complete ---")
        print(f"Final clean master data file has been successfully saved to '{output_path}'.")
    except Exception as e:
        print(f"Error saving file: {e}")
        return None
        
    return final_df

if __name__ == '__main__':
    # Define file paths
    raw_file = 'FS-data-80475.csv'
    output_file = 'cleaned_master_data.csv'
    
    # Execute the final cleaning process
    cleaned_data = definitive_clean_and_aggregate_data(raw_file, output_file)
    
    if cleaned_data is not None:
        print("\n--- First 5 rows of the new, final clean master data: ---")
        print(cleaned_data.head())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def perform_exploratory_analysis(cleaned_data_path):
    """
    Performs exploratory data analysis on the cleaned and aggregated master data.

    Args:
        cleaned_data_path (str): The path to the cleaned_master_data.csv file.
    """
    print("--- Phase 2: Exploratory Data Analysis ---")
    
    # 1. Load the Clean Data
    try:
        df = pd.read_csv(cleaned_data_path, parse_dates=['date'])
        print(f"Successfully loaded clean master data from '{cleaned_data_path}'.")
    except FileNotFoundError:
        print(f"Error: The file '{cleaned_data_path}' was not found. Please run Phase 1 first.")
        return

    # Set plot style for better aesthetics
    sns.set_style("whitegrid")
    plt.rcParams['figure.figsize'] = (15, 7)

    # 2. Identify and Visualize a Key Business KPI
    print("\nStep 1: Visualizing a key business metric...")
    
    # We'll focus on 'TOTAL GROSS PROFIT' as an example. You can change this name.
    kpi_to_plot_name = 'TOTAL GROSS PROFIT' 
    
    kpi_data = df[df['english_name'] == kpi_to_plot_name]

    if kpi_data.empty:
        print(f"Warning: Could not find data for KPI named '{kpi_to_plot_name}'. Please check the name.")
    else:
        plt.figure()
        plt.plot(kpi_data['date'], kpi_data['monthly_value'], marker='o', linestyle='-', color='#0068C9')
        plt.title(f'Monthly Trend for: {kpi_to_plot_name}')
        plt.xlabel('Date')
        plt.ylabel('Monthly Value')
        plt.show()

    # 3. Correlation Analysis for Top KPIs
    print("\nStep 2: Analyzing correlation between top KPIs...")
    
    # Find the top 20 KPIs by average monthly value to keep the heatmap readable
    kpi_summary = df.groupby('english_name')['monthly_value'].mean().reset_index()
    top_20_kpis = kpi_summary.sort_values(by='monthly_value', ascending=False).head(20)['english_name'].tolist()

    # Pivot the data to a 'wide' format for correlation
    df_wide = df[df['english_name'].isin(top_20_kpis)].pivot_table(
        index='date', 
        columns='english_name', 
        values='monthly_value'
    )
    
    # Fill any missing values before calculating correlation
    df_wide.ffill(inplace=True)
    df_wide.bfill(inplace=True)

    # Calculate and plot the correlation matrix
    correlation_matrix = df_wide.corr()

    plt.figure(figsize=(18, 15))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
    plt.title('Correlation Matrix of Top 20 KPIs')
    plt.show()
    
    print("\n--- Phase 2 Complete ---")

if __name__ == '__main__':
    # Define the input file path
    cleaned_file = 'cleaned_master_data.csv'
    
    # Execute the analysis
    perform_exploratory_analysis(cleaned_file)


In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import stopit
import os

# Suppress informational messages from Prophet
warnings.filterwarnings('ignore', category=FutureWarning)

def generate_all_forecasts(cleaned_data_path, output_path, months_to_predict=3, timeout_seconds=45):
    """
    Trains a Prophet model for each unique KPI from the clean master data file.
    This is a fresh, single run with robust error handling.

    Args:
        cleaned_data_path (str): Path to the cleaned_master_data.csv file.
        output_path (str): Path to save the final forecast data.
        months_to_predict (int): The number of months to forecast.
        timeout_seconds (int): Max seconds to wait for a model to train.

    Returns:
        pandas.DataFrame: A DataFrame containing all successful forecasts.
    """
    print("--- Phase 3: Predictive Modeling ---")
    
    # 1. Load the Clean Data
    try:
        df = pd.read_csv(cleaned_data_path, parse_dates=['date'])
        print(f"Successfully loaded clean master data from '{cleaned_data_path}'.")
    except FileNotFoundError:
        print(f"Error: The file '{cleaned_data_path}' was not found. Please run Phase 1 first.")
        return None

    # Start with a fresh forecast file
    if os.path.exists(output_path):
        print(f"Removing old forecast file '{output_path}' to start fresh.")
        os.remove(output_path)

    all_forecasts = []
    unique_kpis = df['english_name'].unique()
    total_kpis = len(unique_kpis)
    print(f"\nFound {total_kpis} unique KPIs to forecast.")

    # 2. Loop Through Each KPI to Train and Forecast
    for i, kpi_name in enumerate(unique_kpis):
        print(f"Processing KPI {i+1}/{total_kpis}: {kpi_name}")
        
        try:
            kpi_data = df[df['english_name'] == kpi_name].copy()
            
            # Prophet requires specific column names: 'ds' for date and 'y' for value
            kpi_data.rename(columns={'date': 'ds', 'monthly_value': 'y'}, inplace=True)
            
            # --- Data Validation ---
            if len(kpi_data) < 12 or kpi_data['y'].nunique() < 2:
                print(f"  -> Skipping '{kpi_name}' due to insufficient data or constant value.")
                continue

            # --- Normalization ---
            y_values = kpi_data['y'].values.reshape(-1, 1)
            scaler = MinMaxScaler(feature_range=(0, 1))
            kpi_data['y_scaled'] = scaler.fit_transform(y_values)

            # --- Model Training with Timeout ---
            model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            
            print(f"  -> Fitting model...")
            with stopit.ThreadingTimeout(timeout_seconds) as context_manager:
                model.fit(kpi_data[['ds', 'y_scaled']].rename(columns={'y_scaled': 'y'}))
            
            if context_manager.state != stopit.ThreadingTimeout.EXECUTED:
                print(f"  -> TIMEOUT: Model fitting for '{kpi_name}' took too long. Skipping.")
                continue
            
            print(f"  -> Model fitting complete.")
            
            # --- Prediction and Rescaling ---
            future_dates = model.make_future_dataframe(periods=months_to_predict, freq='MS')
            forecast = model.predict(future_dates)
            
            predicted_values_scaled = forecast[['yhat', 'yhat_lower', 'yhat_upper']].values
            forecast[['yhat', 'yhat_lower', 'yhat_upper']] = scaler.inverse_transform(predicted_values_scaled)

            # --- Store Results ---
            forecast['english_name'] = kpi_name
            results = forecast[['ds', 'english_name', 'yhat', 'yhat_lower', 'yhat_upper']]
            results = results.merge(kpi_data[['ds', 'y']], on='ds', how='left')
            
            all_forecasts.append(results)
        
        except Exception as e:
            print(f"  -> FAILED to process '{kpi_name}'. Error: {e}. Skipping.")
            continue

    if not all_forecasts:
        print("\nNo forecasts could be generated.")
        return None

    # 3. Combine and Save Final Forecast File
    final_forecast_df = pd.concat(all_forecasts)
    final_forecast_df.rename(columns={
        'ds': 'date', 
        'y': 'actual_value', 
        'yhat': 'predicted_value'
    }, inplace=True)
    
    try:
        final_forecast_df.to_csv(output_path, index=False)
        print(f"\n--- Phase 3 Complete ---")
        print(f"Forecast master data file saved successfully to '{output_path}'.")
    except Exception as e:
        print(f"Error saving file: {e}")
        return None

    return final_forecast_df

if __name__ == '__main__':
    # Define file paths
    cleaned_file = 'cleaned_master_data.csv'
    output_file = 'forecast_master_data.csv'
    
    # Execute the forecasting process
    forecast_data = generate_all_forecasts(cleaned_file, output_file)
    
    if forecast_data is not None:
        print("\n--- First 5 rows of the new forecast master data: ---")
        print(forecast_data.head())
